In [1]:
import pandas as pd
import numpy as np

In [ ]:
import subprocess
import time
import os
from IPython.display import display, Markdown

# --- 1. ตรวจสอบและแสดงสถานะ GPU ---
# **คำแนะนำ:** กรุณาตรวจสอบให้แน่ใจว่าคุณได้เลือก "Runtime type" เป็น GPU (เช่น T4, A100 หรือ V100) แล้ว
display(Markdown("## 1. ตรวจสอบสถานะ GPU"))
try:
    print("Running nvidia-smi to check GPU status...")
    subprocess.run("nvidia-smi", shell=True, check=True)
except subprocess.CalledProcessError:
    print("\n[🚨 WARNING] Could not run nvidia-smi. Please ensure your Colab runtime is set to GPU.")

# --- 2. ติดตั้งเครื่องมือและ Ollama ---
display(Markdown("\n## 2. ติดตั้ง Ollama และเครื่องมือที่จำเป็น"))

# ติดตั้ง lshw ตามที่ร้องขอ
print("Installing lshw...")
subprocess.run("apt-get install -y lshw > /dev/null 2>&1", shell=True, check=True)
subprocess.run("lshw -C display", shell=True)

# ติดตั้ง Ollama
print("\nInstalling Ollama...")
# สคริปต์ติดตั้ง Ollama
ollama_install_script = """
curl -fsSL https://ollama.com/install.sh | sh
"""
subprocess.run(ollama_install_script, shell=True, check=True)
print("Ollama installed successfully.")

# --- 3. เริ่มต้น Ollama Server และรอให้พร้อมใช้งาน ---
display(Markdown("\n## 3. เริ่มต้น Ollama Server"))

# รัน Ollama server ใน background
# ใช้ nohup และ & เพื่อรันเป็น background process ใน Colab
print("Starting Ollama server in background...")
os.system("nohup ollama serve > ollama.log 2>&1 &")

# ให้เวลากับ server ในการเริ่มต้น
print("Waiting 15 seconds for Ollama server to initialize...")
time.sleep(15)

# ตรวจสอบว่า server รันอยู่หรือไม่
try:
    print("\nChecking server status...")
    subprocess.run("ollama list", shell=True, check=True)
    print("Ollama server is running and accessible.")
except subprocess.CalledProcessError:
    print("\n[🚨 ERROR] Ollama server failed to start or respond. Check 'ollama.log'.")
    exit()

# --- 4. ดึง (Pull) โมเดลที่ต้องการ ---
display(Markdown("\n## 4. ดึง (Pull) โมเดล Llama3 และ Typhoon"))

# ดึง Llama3
print("\nPulling llama3 (ประมาณ 4.7 GB)...")
subprocess.run("ollama pull llama3", shell=True, check=True)

# ดึง scb10x/typhoon2.1-gemma3-4b:latest (โมเดลภาษาไทย)
print("\nPulling scb10x/typhoon2.1-gemma3-4b:latest...")
subprocess.run("ollama pull scb10x/typhoon2.1-gemma3-4b:latest", shell=True, check=True)

display(Markdown("\n## 5. ทดสอบการรันโมเดล"))
print("--- Running test prompt with llama3 ---")
# ทดสอบรันโมเดลเพื่อให้แน่ใจว่าใช้งานได้
test_prompt = "Explain why the sky is blue in one sentence."
try:
    # ใช้ -q เพื่อไม่แสดงข้อมูลระหว่างการดาวน์โหลด/รัน
    subprocess.run(f'ollama run llama3 "{test_prompt}"', shell=True, check=True)
    print("\n✅ Setup Complete! Both models are pulled and ready for use.")
except subprocess.CalledProcessError as e:
    print(f"\n[🚨 FATAL ERROR] Test run failed. Error: {e}")

display(Markdown("\n---"))
display(Markdown("### วิธีการใช้งานต่อ:"))
display(Markdown("คุณสามารถเรียกใช้โมเดลในเซลล์ใหม่ได้ทันที เช่น:"))
display(Markdown("`!ollama run llama3 \"What is the capital of France?\"`"))
display(Markdown("`!ollama run scb10x/typhoon2.1-gemma3-4b:latest \"เขียนบทกลอนสั้น ๆ เกี่ยวกับประเทศไทย\"`"))


In [2]:
hotel = pd.read_csv('https://raw.githubusercontent.com/Thanrada2024/illegal_accommodation/refs/heads/main/bora/hotel_bangkok.csv').iloc[:,[0,1,3,4]]
hotel.columns = ['Hotel_Ref','Hotel_Name','Hotel_Address','N']
hotel["Hotel_Address"]= hotel["Hotel_Address"].replace("บ้านเลขที่ ","", regex=True)
hotel.head()

,Hotel_Ref,Hotel_Name,Hotel_Address,N
0,101/2563,โรงแรม สุทธิสารพ้อยส์,45 ซ.ยิ้มอุปถัมภ์ แขวงรัชดาภิเษก เขตดินแดง จ.ก...,42
1,136/2563,โรงแรม คณาบูทีค 2,"3,5,7,9,11 ซ.จรัญสนิทวงศ์ 57/2 แขวงบางบำหรุ เข...",24
2,152/2563,วัลย์ลดา เพลส,71/1 ซ.รามคำแหง 53 (จันทร์ศรีชวาลา) แขวงพลับพล...,50
3,214/2563,โรงแรม นิวสยาม พาเลซวิลล์ 2,56 ตรอกโรงไหม แขวงชนะสงคราม เขตพระนคร จ.กรุงเท...,48
4,215/2563,โรงแรม นิวสยาม พาเลซวิลล์ 3,58 ตรอกโรงไหม แขวงชนะสงคราม เขตพระนคร จ.กรุงเท...,48


In [3]:
trip_com = pd.read_csv('https://raw.githubusercontent.com/Thanrada2024/illegal_accommodation/refs/heads/main/OTA/trip_com_bkk.csv')[['name','name href','address']].dropna()
trip_com['OTA'] = 'trip_com'

booking_com = pd.read_csv('https://raw.githubusercontent.com/Thanrada2024/illegal_accommodation/main/OTA/booking_com_bkk_with_addr.csv', engine='python')[['name href','name','address']].dropna()
booking_com['OTA'] = 'booking_com'

# รวมสอง DataFrame เข้าด้วยกัน
OTA_df = pd.concat([trip_com, booking_com], ignore_index=True)
OTA_df

,name,name href,address,OTA
0,นาซ่า กรุงเทพฯ,https://th.trip.com/hotels/detail/?cityId=359&...,"44 ถ. สุขุมวิท 71 แขวงสวนหลวง, เขตสวนหลวง, กรุ...",trip_com
1,โรงแรมอีสติน แกรนด์ พญาไท,https://th.trip.com/hotels/detail/?cityId=359&...,"18 ถ.พญาไท ทุ่งพญาไท, เขตราชเทวี, กรุงเทพฯ, 10400",trip_com
2,โรงแรมเอเชียกรุงเทพ,https://th.trip.com/hotels/detail/?cityId=359&...,"269 ถ.พ, เขตราชเทวี, กรุงเทพฯ, 10400",trip_com
3,โรงแรมทวิน ทาวเวอร์,https://th.trip.com/hotels/detail/?cityId=359&...,"88 รองเมือง ปทุมวัน, ถ.พระราม 6, เขตปทุมวัน, ก...",trip_com
4,โรงแรมเดอะ เบอร์เคลีย์ ประตูน้ำ,https://th.trip.com/hotels/detail/?cityId=359&...,"559 ราชปรารภ, มักกะสัน ราชเทวี, เขตราชเทวี, กร...",trip_com
...,...,...,...,...
2660,บรรทัดทองโฮสเทล,https://www.booking.com/hotel/th/banthat-thong...,"1806/16-17 ถ.บรรทัดทอง รองเมือง ปทุมวัน, ปทุมว...",booking_com
2661,Vaeh Viean Silom,https://www.booking.com/hotel/th/vaeh-viean-ho...,"23 Pramot Alley, บางรัก, 10500 กรุงเทพมหานคร, ไทย",booking_com
2662,พาราดิโซบูติคสวีท,https://www.booking.com/hotel/th/paradiso-bout...,"1/11-12 สุขุมวิท 10, คลองเตย, 10110 กรุงเทพมหา...",booking_com
2663,Silom Space Hostel,https://www.booking.com/hotel/th/silom-space-h...,"8 Silom Road Soi 2, Bangrak, บางรัก, 10500 กรุ...",booking_com


## Method 1: Semantic search

In [13]:
# ---------- ตั้งค่า (from 614a1f16) ----------
import re, unicodedata
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.neighbors import NearestNeighbors
from tqdm.auto import tqdm # Import tqdm for progress bars

# =========================
# 0) เตรียมตารางต้นทาง (from 614a1f16)
# =========================
# trip_com: ต้องมีคอลัมน์ ['name', 'name href', 'address']
trip = OTA_df.rename(columns={
    "name": "Trip_Name",
    "name href": "Trip_URL",
    "address": "Trip_Address"
}).copy()

# hotel_list: ต้องมีคอลัมน์ชื่อ/ที่อยู่ตามตัวอย่างในภาพ
hotels = hotel.copy()
if "v-data-table__td 2" in hotels.columns:
    hotels = hotels.rename(columns={"v-data-table__td 2": "Hotel_Name"})
if "v-data-table__td 4" in hotels.columns:
    hotels = hotels.rename(columns={"v-data-table__td 4": "Hotel_Address"})
if "v-data-table__td" in hotels.columns:
    hotels["Hotel_Ref"] = hotels["v-data-table__td"]  # เลขอ้างอิงเอกสาร/ไอดี

for c in ["Trip_Name","Trip_Address"]:
    if c not in trip.columns: trip[c] = ""
trip[["Trip_Name","Trip_Address"]] = trip[["Trip_Name","Trip_Address"]].fillna("")

for c in ["Hotel_Name","Hotel_Address"]:
    if c not in hotels.columns: hotels[c] = ""
hotels[["Hotel_Name","Hotel_Address"]] = hotels[["Hotel_Name","Hotel_Address"]].fillna("")

# =========================
# 1) Clean: ชื่อ/ที่อยู่ + สร้าง "คีย์" name+address (from 614a1f16)
# =========================
def normalize_name(s: str) -> str:
    """ตัดคำ 'โรงแรม/Hotel' และ 'กรุงเทพ/กรุงเทพฯ/กรุงเทพมหานคร' + ลบเว้นวรรค/สัญลักษณ์"""
    if pd.isna(s): return ""
    s = unicodedata.normalize("NFKC", str(s)).lower()
    s = re.sub(r"(โรงแรม|hotel)", " ", s, flags=re.IGNORECASE)
    s = re.sub(r"กรุงเทพ(?:มหานคร|ฯ)?", " ", s)  # ตัดคำ 'กรุงเทพ' ออกจาก 'ชื่อ'
    s = re.sub(r"[,\|\(\)\[\]{}\"'’`•·/.:;#&\-–—_+*^@!$?~]", " ", s)
    s = re.sub(r"\s+", "", s)  # ลบช่องว่างทั้งหมด
    return s

def normalize_addr(s: str) -> str:
    """คงโครงสร้างที่อยู่ไว้ (ไม่ลบช่องว่างทั้งหมด) แต่ normalize ให้เท่า ๆ กัน"""
    if pd.isna(s): return ""
    s = unicodedata.normalize("NFKC", str(s)).lower().strip()

    # ทำให้คำหลักสอดคล้องกัน
    rep = {
        "กทม.": "กรุงเทพมหานคร",
        "กรุงเทพฯ": "กรุงเทพมหานคร",
        "จ.": "",              # จ.กรุงเทพมหานคร -> กรุงเทพมหานคร
        "เขต ": "",
        "แขวง ": "",
        "ถ.": "",
        "ถนน ": "",        # กันกรณีมีช่องว่างเพี้ยน
        "บ้านเลขที่": "",
    }
    for k,v in rep.items():
        s = s.replace(k, v)

    # เก็บตัวอักษร/ตัวเลข/เว้นวรรคและเครื่องหมาย / , - บางส่วน
    s = re.sub(r"[\"'’`•·\|\[\]{}#@!$^*?_+=:;,]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return "".join(s.split())

# apply clean
trip["Trip_Name_n"]    = trip["Trip_Name"].map(normalize_name)
trip["Trip_Address_n"] = trip["Trip_Address"].map(normalize_addr)
hotels["Hotel_Name_n"]    = hotels["Hotel_Name"].map(normalize_name)
hotels["Hotel_Address_n"] = hotels["Hotel_Address"].map(normalize_addr)

# คีย์ค้นหา = name + address (ตามที่ต้องการ)
trip["Trip_Key"]   = (trip["Trip_Name_n"]   + " " + trip["Trip_Address_n"]).str.strip()
hotels["Hotel_Key"] = (hotels["Hotel_Name_n"] + " " + hotels["Hotel_Address_n"]).str.strip()

# =========================
# 2) Embeddings (from 614a1f16)
# =========================
# Using sentence-transformers as it was successful previously
MODEL = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
model = SentenceTransformer(MODEL)

# embeddings แยก (สำหรับรายงาน sim_name / sim_addr) + คีย์รวม (สำหรับค้นหา)
emb_trip_name  = model.encode(trip["Trip_Name_n"].tolist(),   convert_to_numpy=True, normalize_embeddings=True, batch_size=128, show_progress_bar=True)
emb_trip_addr  = model.encode(trip["Trip_Address_n"].tolist(), convert_to_numpy=True, normalize_embeddings=True, batch_size=128, show_progress_bar=True)
emb_trip_key   = model.encode(trip["Trip_Key"].tolist(),      convert_to_numpy=True, normalize_embeddings=True, batch_size=128, show_progress_bar=True)

emb_hotel_name = model.encode(hotels["Hotel_Name_n"].tolist(),   convert_to_numpy=True, normalize_embeddings=True, batch_size=128, show_progress_bar=True)
emb_hotel_addr = model.encode(hotels["Hotel_Address_n"].tolist(), convert_to_numpy=True, normalize_embeddings=True, batch_size=128, show_progress_bar=True)
emb_hotel_key  = model.encode(hotels["Hotel_Key"].tolist(),      convert_to_numpy=True, normalize_embeddings=True, batch_size=128, show_progress_bar=True)


# =========================
# 3) ค้นหาคู่โดยใช้ "คีย์รวม" แล้วคำนวณ sim_name/sim_addr แยก (from 614a1f16)
# =========================
# ใช้ NearestNeighbors เพื่อสเกลกับข้อมูลใหญ่ และใช้ cosine distance
nn = NearestNeighbors(n_neighbors=1, metric="cosine")
nn.fit(emb_hotel_key)
dist, idx = nn.kneighbors(emb_trip_key, return_distance=True)
sim_key = 1.0 - dist.ravel()        # cosine similarity
best_idx = idx.ravel()

# similarity แยกชื่อ/ที่อยู่ กดดอทกับคู่ที่เลือก (embedding ถูก normalize แล้ว)
best_name = np.einsum("ij,ij->i", emb_trip_name, emb_hotel_name[best_idx])
best_addr = np.einsum("ij,ij->i", emb_trip_addr, emb_hotel_addr[best_idx])

# เกณฑ์ผ่านหลักจากคีย์รวม (name+address)
KEY_THRESH = 0.95   # ปรับตามการแจกแจงของข้อมูลคุณ
is_match = (sim_key >= KEY_THRESH)


# =========================
# 4) สร้างผลลัพธ์ โดยยึด trip_com เป็นหลัก (from 614a1f16)
# =========================
matched_hotels = hotels.iloc[best_idx].reset_index(drop=True)

trip_match = trip.copy()
trip_match["Match_Hotel_Name"]    = np.where(is_match, matched_hotels["Hotel_Name"].values,    np.nan)
trip_match["Match_Hotel_Address"] = np.where(is_match, matched_hotels["Hotel_Address"].values, np.nan)
trip_match["Match_Hotel_Ref"]     = np.where(
    is_match,
    matched_hotels.get("Hotel_Ref", pd.Series([np.nan]*len(trip))).values,
    np.nan
)

# similarity แยก + รวม
trip_match["sim_name"] = best_name
trip_match["sim_addr"] = best_addr
trip_match["sim_key"]  = sim_key     # ความคล้ายของคีย์ name+address
trip_match["is_match"] = is_match

# 4.1 เติม best-candidate ให้ครบทุกแถว
trip_match["Best_Hotel_Name"]    = matched_hotels["Hotel_Name"].values
trip_match["Best_Hotel_Address"] = matched_hotels["Hotel_Address"].values
trip_match["Best_Hotel_Ref"]     = matched_hotels.get(
    "Hotel_Ref", pd.Series([np.nan]*len(trip_match))
).values
# Add the best matching hotel key
trip_match["Best_Hotel_Key"]     = matched_hotels["Hotel_Key"].values


# 4.2 คง Match_* สำหรับเฉพาะแถวที่ผ่านเกณฑ์ (ตามโค้ดเดิม)
trip_match["Match_Hotel_Name"]    = np.where(is_match, matched_hotels["Hotel_Name"].values,    np.nan)
trip_match["Match_Hotel_Address"] = np.where(is_match, matched_hotels["Hotel_Address"].values, np.nan)
trip_match["Match_Hotel_Ref"]     = np.where(is_match, matched_hotels.get("Hotel_Ref", pd.Series([np.nan]*len(trip_match))).values, np.nan)

# สรุปภาพรวม semantic search
total_semantic   = len(trip_match)
matched_semantic = int(trip_match["is_match"].sum())
print(f"Semantic search matched {matched_semantic}/{total_semantic} rows ({matched_semantic/total_semantic:.1%}) with KEY_THRESH={KEY_THRESH}")


# ---------- LLM Verification (from 56128e67) ----------
import os, requests
from requests.adapters import HTTPAdapter, Retry
import subprocess
import time

OLLAMA_URL = os.getenv("OLLAMA_URL", "http://127.0.0.1:11434")
VERIFY_MODEL = "scb10x/typhoon2.1-gemma3-4b:latest" # Or another suitable LLM model

# robust HTTP session with retries
_session = requests.Session()
retries = Retry(total=5, backoff_factor=0.5, status_forcelist=[429,500,502,503,504])
_session.mount("http://", HTTPAdapter(max_retries=retries))

# Ensure Ollama server is running and model is pulled
try:
    subprocess.run("ollama list", shell=True, check=True, capture_output=True, text=True)
    print("Ollama server is running.")
except subprocess.CalledProcessError:
    print("Starting Ollama server in background...")
    # Use nohup and & to ensure it runs in the background even if the cell output is cleared
    get_ipython().system_raw('nohup ollama serve > ollama_verify.log 2>&1 &')
    # Give Ollama time to start
    print("Waiting 15 seconds for Ollama server to initialize...")
    time.sleep(15)
    try:
        subprocess.run("ollama list", shell=True, check=True, capture_output=True, text=True)
        print("Ollama server started successfully.")
    except subprocess.CalledProcessError:
        print("[FATAL ERROR] Ollama server failed to start. LLM verification will not work.")


print(f"Pulling {VERIFY_MODEL}...")
try:
    subprocess.run(f"ollama list | grep {VERIFY_MODEL}", shell=True, check=True, capture_output=True, text=True)
    print(f"{VERIFY_MODEL} is already pulled.")
except subprocess.CalledProcessError:
    print(f"Pulling {VERIFY_MODEL}...")
    pull_process = subprocess.run(f"ollama pull {VERIFY_MODEL}", shell=True, capture_output=True, text=True)
    if pull_process.returncode != 0:
        print(f"[ERROR] Failed to pull {VERIFY_MODEL}. Error: {pull_process.stderr}")
        # Decide how to handle fatal error
    else:
        print(f"Pulled {VERIFY_MODEL}.")


# Prompt template for LLM verification
_VERIFY_PROMPT = """พิจารณาข้อมูลสองรายการต่อไปนี้:
รายการ A: "{key_a}"
รายการ B: "{key_b}"

จากชื่อและที่อยู่ (ที่รวมอยู่ในคีย์ข้างต้น) จงตัดสินว่ารายการ A และ B อ้างถึง "สถานที่เดียวกัน" หรือไม่
ให้ตอบเป็น JSON ก้อนเดียวเท่านั้น ในรูปแบบ:
{{
  "is_same_place": true // หรือ false
}}

กติกา:
- ตอบ JSON เท่านั้น ห้ามมีข้อความอื่นใดนอก JSON
- ให้ค่า "is_same_place" เป็น boolean (true/false)
- พิจารณาความหมายและบริบท แม้การสะกดหรือรูปแบบจะต่างกันเล็กน้อย

"""

def _json_only(s: str):
    s = s.strip()
    i, j = s.find("{"), s.rfind("}")
    if i == -1 or j == -1 or j <= i: return None
    try: return json.loads(s[i:j+1])
    except json.JSONDecodeError: return None

def llm_verify_keys(key_a: str, key_b: str, timeout=60) -> bool | None:
    """
    Verify if two keys represent the same place using an LLM.
    Returns True, False, or None if verification fails.
    """
    if not key_a or not key_b:
        return None # Cannot verify with empty keys

    prompt = _VERIFY_PROMPT.format(key_a=key_a, key_b=key_b)

    payload = {
        "model": VERIFY_MODEL,
        "prompt": prompt,
        "stream": False,
        "options": {"temperature": 0.0}
        # "format": "json" # Use if your Ollama version supports it
    }

    try:
        r = _session.post(f"{OLLAMA_URL}/api/generate", json=payload, timeout=timeout)
        r.raise_for_status()
        resp_text = r.json().get("response","").strip()
        data = _json_only(resp_text)

        if isinstance(data, dict) and "is_same_place" in data:
            return bool(data["is_same_place"])
        else:
            print(f"Warning: LLM did not return valid boolean for keys '{key_a[:30]}...' and '{key_b[:30]}...'. Response: {resp_text[:100]}...")
            return None # LLM response not in expected format

    except requests.exceptions.RequestException as e:
        print(f"Error calling LLM for keys '{key_a[:30]}...' and '{key_b[:30]}...': {e}")
        return None # API call failed
    except Exception as e:
        print(f"Unexpected error during LLM verification for keys '{key_a[:30]}...' and '{key_b[:30]}...': {e}")
        return None # Other unexpected errors


# ---------- Apply LLM verification to trip_match DataFrame ----------

# Select rows to verify: e.g., those identified as potential matches or borderline cases
# For simplicity, let's verify all rows where a Best_Hotel_Key was found
# Filter based on semantic search matches or potential matches (e.g., sim_key > 0.85)
verify_df = trip_match[trip_match['sim_key'] >= 0.85].copy() # Example: verify rows with sim_key >= 0.85

# Add a new column for LLM verification result
verify_df['llm_verified_same'] = None # Initialize with None

print(f"\nVerifying {len(verify_df)} potential matches with LLM...")

# Iterate and verify
for index, row in tqdm(verify_df.iterrows(), total=len(verify_df), desc="LLM Verifying Matches"):
    trip_key = row['Trip_Key']
    best_hotel_key = row['Best_Hotel_Key']

    # Only verify if both keys are non-empty
    if trip_key and best_hotel_key:
        is_same = llm_verify_keys(trip_key, best_hotel_key)
        # Update the original trip_match DataFrame with the result
        # Use .loc for reliable assignment based on original index
        trip_match.loc[index, 'llm_verified_same'] = is_same
    else:
        # If keys are empty, LLM can't verify, set to None
        trip_match.loc[index, 'llm_verified_same'] = None


print("\nLLM verification complete.")

# Display results, focusing on relevant columns including the new one
display(trip_match[['Trip_Name', 'Trip_Address', 'Trip_Key',
                    'Best_Hotel_Name', 'Best_Hotel_Address', 'Best_Hotel_Ref', 'Best_Hotel_Key',
                    'sim_name', 'sim_addr', 'sim_key', 'is_match', 'llm_verified_same']].head(20))

# Optional: Summarize verification results
print("\nLLM Verification Summary:")
display(trip_match['llm_verified_same'].value_counts(dropna=False))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Batches:   0%|          | 0/21 [00:00<?, ?it/s]

Batches:   0%|          | 0/21 [00:00<?, ?it/s]

Batches:   0%|          | 0/21 [00:00<?, ?it/s]

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

...

LLM Verifying Matches:   0%|          | 0/1716 [00:00<?, ?it/s]


LLM verification complete.


,Trip_Name,Trip_Address,Trip_Key,Best_Hotel_Name,Best_Hotel_Address,Best_Hotel_Ref,Best_Hotel_Key,sim_name,sim_addr,sim_key,is_match,llm_verified_same
0,นาซ่า กรุงเทพฯ,"44 ถ. สุขุมวิท 71 แขวงสวนหลวง, เขตสวนหลวง, กรุ...",นาซ่า 44สุขุมวิท71แขวงสวนหลวงเขตสวนหลวงกรุงเทพ...,โรงแรมเบสท์เวสเทิร์น นาดา ดอนเมือง แอร์พอร์ต,235/8 ถ.พหลโยธิน แขวงอนุสาวรีย์ เขตบางเขน จ.กร...,150/2565,เบสท์เวสเทิร์นนาดาดอนเมืองแอร์พอร์ต 235/8พหลโย...,0.416311,0.768178,0.758598,False,NaN
1,โรงแรมอีสติน แกรนด์ พญาไท,"18 ถ.พญาไท ทุ่งพญาไท, เขตราชเทวี, กรุงเทพฯ, 10400",อีสตินแกรนด์พญาไท 18พญาไททุ่งพญาไทเขตราชเทวีกร...,โรงแรม อีสติน แกรนด์ พญาไท,18 ถ.พญาไท แขวงทุ่งพญาไท เขตราชเทวี จ.กรุงเทพม...,19/2566,อีสตินแกรนด์พญาไท 18พญาไทแขวงทุ่งพญาไทเขตราชเท...,1.000000,0.966716,0.990144,True,True
2,โรงแรมเอเชียกรุงเทพ,"269 ถ.พ, เขตราชเทวี, กรุงเทพฯ, 10400",เอเชีย 269พเขตราชเทวีกรุงเทพมหานคร10400,เอเชีย,296 ถ.พญาไท แขวงถนนเพชรบุรี เขตราชเทวี จ.กรุงเ...,107/2564,เอเชีย 296พญาไทแขวงถนนเพชรบุรีเขตราชเทวีกรุงเท...,1.000000,0.905918,0.923781,False,True
3,โรงแรมทวิน ทาวเวอร์,"88 รองเมือง ปทุมวัน, ถ.พระราม 6, เขตปทุมวัน, ก...",ทวินทาวเวอร์ 88รองเมืองปทุมวันพระราม6เขตปทุมวั...,เดอะ ทวิน ทาวเวอร์ กรุงเทพฯ,88 ซ.รองเมือง 2 แขวงรองเมือง เขตปทุมวัน จ.กรุง...,74/2564,เดอะทวินทาวเวอร์ 88ซ.รองเมือง2แขวงรองเมืองเขตป...,0.929925,0.972836,0.971434,True,True
4,โรงแรมเดอะ เบอร์เคลีย์ ประตูน้ำ,"559 ราชปรารภ, มักกะสัน ราชเทวี, เขตราชเทวี, กร...",เดอะเบอร์เคลีย์ประตูน้ํา 559ราชปรารภมักกะสันรา...,เดอะ เบอร์เคลีย์ โฮเต็ล ประตูน้ำ,559 ถ.ราชปรารภ แขวงมักกะสัน เขตราชเทวี จ.กรุงเ...,194/2566,เดอะเบอร์เคลีย์โฮเต็ลประตูน้ํา 559ราชปรารภแขวง...,0.966134,0.966027,0.971411,True,True
5,12 เดอะ เรสซิเดนซ์ โฮเทลแอนด์อพาร์ตเมนต์,"22 ซ.สรณคมน์ 12 แขวงสีกัน, เขตดอนเมือง, กรุงเท...",12เดอะเรสซิเดนซ์โฮเทลแอนด์อพาร์ตเมนต์ 22ซ.สรณค...,โรงแรม เคที เกสต์เฮาส์,14 ซ.อินทามระ 44 แขวงรัชดาภิเษก เขตดินแดง จ.กร...,117/2565,เคทีเกสต์เฮาส์ 14ซ.อินทามระ44แขวงรัชดาภิเษกเขต...,0.776499,0.958448,0.874566,False,False
6,ไอบิส สไตล์ กรุงเทพ สีลม,"289 ถ.สีลม, เขตบางรัก, กรุงเทพฯ, 10500",ไอบิสสไตล์สีลม 289สีลมเขตบางรักกรุงเทพมหานคร10500,ไอบิส สไตล์ กรุงเทพ สีลม,289 ถ.สีลม แขวงสีลม เขตบางรัก จ.กรุงเทพมหานคร ...,162/2563,ไอบิสสไตล์สีลม 289สีลมแขวงสีลมเขตบางรักกรุงเทพ...,1.000000,0.975502,0.984055,True,True
7,AETAS ลุมพินี,"1030 4 ถ. พระรามที่ ๔ แขวงท, เขตสาทร, กรุงเทพฯ...",aetasลุมพินี 10304พระรามที่๔แขวงทเขตสาทรกรุงเท...,พระโขนง,58 ซ.สุขุมวิท 46 ถ.พระราม 4 แขวงพระโขนง เขตคลอ...,120/2564,พระโขนง 58ซ.สุขุมวิท46พระราม4แขวงพระโขนงเขตคลอ...,0.760066,0.881806,0.914838,False,False
8,เมอร์เคียว กรุงเทพ สุรวงศ์,"222 ถนนสุรวงศ์ แขวงสี่พระยา, เขตบางรัก, กรุงเท...",เมอร์เคียวสุรวงศ์ 222ถนนสุรวงศ์แขวงสี่พระยาเขต...,โรงแรมเมอร์เคียว กรุงเทพ สุรวงศ์,222 ถ.สุรวงศ์ แขวงสี่พระยา เขตบางรัก จ.กรุงเทพ...,56/2567,เมอร์เคียวสุรวงศ์ 222สุรวงศ์แขวงสี่พระยาเขตบาง...,1.000000,0.867777,0.872151,False,True
9,ชาเทรียม แกรนด์ กรุงเทพฯ,"728 ถ., เขตราชเทวี, กรุงเทพฯ, 10400",ชาเทรียมแกรนด์ 728เขตราชเทวีกรุงเทพมหานคร10400,โรงแรม ชาเทรียม แกรนด์ กรุงเทพฯ,728 ถ.เพชรบุรี แขวงถนนเพชรบุรี เขตราชเทวี จ.กร...,142/2568,ชาเทรียมแกรนด์ 728เพชรบุรีแขวงถนนเพชรบุรีเขตรา...,1.000000,0.861601,0.897400,False,True



LLM Verification Summary:


,count
llm_verified_same,
True,976
NaN,949
False,740


In [6]:
trip_match.is_match.value_counts()

,count
is_match,
False,2028
True,637


## Method 2: Ollama

## 1. ตรวจสอบสถานะ GPU

Running nvidia-smi to check GPU status...



## 2. ติดตั้ง Ollama และเครื่องมือที่จำเป็น

Installing lshw...

Installing Ollama...
Ollama installed successfully.



## 3. เริ่มต้น Ollama Server

...


---

### วิธีการใช้งานต่อ:

คุณสามารถเรียกใช้โมเดลในเซลล์ใหม่ได้ทันที เช่น:

`!ollama run llama3 "What is the capital of France?"`

`!ollama run scb10x/typhoon2.1-gemma3-4b:latest "เขียนบทกลอนสั้น ๆ เกี่ยวกับประเทศไทย"`

In [8]:
# ---------- Setup ----------
import re, unicodedata, json, math
import numpy as np
import pandas as pd
import requests
from sklearn.neighbors import NearestNeighbors
from tqdm.auto import tqdm
import time
import subprocess

# Start Ollama server with GPU acceleration (ensure you have a GPU runtime)
# Check if already running to avoid errors
try:
    subprocess.run("ollama list", shell=True, check=True, capture_output=True, text=True)
    print("Ollama server is already running.")
except subprocess.CalledProcessError:
    print("Starting Ollama server in background with GPU acceleration...")
    # Use nohup and & to ensure it runs in the background even if the cell output is cleared
    get_ipython().system_raw('nohup ollama serve --accel gpu > ollama.log 2>&1 &')
    # Give Ollama time to start
    print("Waiting 15 seconds for Ollama server to initialize...")
    time.sleep(15)
    try:
        subprocess.run("ollama list", shell=True, check=True, capture_output=True, text=True)
        print("Ollama server started successfully.")
    except subprocess.CalledProcessError:
        print("[ERROR] Failed to start Ollama server. Check ollama.log for details.")


# Pull the embedding model if not already present
EMB_MODEL = "bge-m3" # Changed embedding model to bge-m3
print(f"Pulling {EMB_MODEL}...")
try:
    subprocess.run(f"ollama list | grep {EMB_MODEL}", shell=True, check=True, capture_output=True, text=True)
    print(f"{EMB_MODEL} is already pulled.")
except subprocess.CalledProcessError:
    print(f"Pulling {EMB_MODEL}...")
    pull_process = subprocess.run(f"ollama pull {EMB_MODEL}", shell=True, capture_output=True, text=True)
    if pull_process.returncode != 0:
        print(f"[ERROR] Failed to pull {EMB_MODEL}. Error: {pull_process.stderr}")
        # Exit or handle the error appropriately if the model is essential
    else:
        print(f"Pulled {EMB_MODEL}.")


# Ollama embeddings function
OLLAMA_URL = "http://localhost:11434"

def ollama_embed(texts, model=EMB_MODEL):
    embs = []
    valid_texts = []
    for t in tqdm(texts, desc=f"Embedding with {model}"):
        try:
            # Ensure text is not empty or just whitespace
            if not t or not str(t).strip():
                embs.append(None) # Append None for empty/whitespace texts
                continue

            resp = requests.post(
                f"{OLLAMA_URL}/api/embeddings",
                json={"model": model, "input": str(t)}, # Ensure input is string
                timeout=60
            )
            resp.raise_for_status()
            data = resp.json()

            if data and 'embedding' in data and data['embedding']:
                v = np.array(data["embedding"], dtype=np.float32)
                if v.size > 0:
                    n = np.linalg.norm(v)
                    if n > 0:
                        v = v / n
                    embs.append(v)
                    valid_texts.append(t)
                else:
                     print(f"Warning: Received empty embedding array for text: {t[:50]}...")
                     embs.append(None) # Append None for empty embedding array
            else:
                print(f"Warning: Received no embedding data for text: {t[:50]}... Response: {data}")
                embs.append(None) # Append None for no embedding data


        except requests.exceptions.RequestException as e:
            print(f"Error embedding text: {t[:50]}... Error: {e}")
            embs.append(None) # Append None on request error
        except Exception as e:
             print(f"Unexpected error embedding text: {t[:50]}... Error: {e}")
             embs.append(None)


    # Filter out None values and convert to numpy array
    valid_embs = [e for e in embs if e is not None]

    if not valid_embs:
        print(f"Warning: No valid embeddings generated for model {model}")
        return np.empty((0, 0), dtype=np.float32), [] # Return empty array and empty list of valid texts


    # Ensure all valid embeddings have the same dimension
    expected_dimension = valid_embs[0].shape[0]
    final_embs = []
    final_texts = []
    for i, emb in enumerate(embs):
        if emb is not None and emb.shape[0] == expected_dimension:
            final_embs.append(emb)
            final_texts.append(texts[i])
        else:
            # Handle cases where embedding was None or had wrong dimension
            # You might want to log this or handle differently
            pass # Skipping texts that didn't produce valid embeddings


    if not final_embs:
         print(f"Warning: No valid embeddings with consistent dimension generated for model {model}")
         return np.empty((0, 0), dtype=np.float32), []


    return np.vstack(final_embs), final_texts


# Load and prepare data
trip = OTA_df.rename(columns={"name": "Trip_Name", "name href": "Trip_URL", "address": "Trip_Address"}).copy()
hotels = hotel.copy()
if "v-data-table__td 2" in hotels.columns: hotels = hotels.rename(columns={"v-data-table__td 2": "Hotel_Name"})
if "v-data-table__td 4" in hotels.columns: hotels = hotels.rename(columns={"v-data-table__td 4": "Hotel_Address"})
if "v-data-table__td" in hotels.columns: hotels["Hotel_Ref"] = hotels["v-data-table__td"]

for df_ in [trip, hotels]:
    for col in ["Trip_Name", "Trip_Address", "Hotel_Name", "Hotel_Address"]:
        if col in df_.columns:
            df_[col] = df_[col].fillna("")

# Clean names and addresses
def normalize_name(s: str) -> str:
    if pd.isna(s): return ""
    s = unicodedata.normalize("NFKC", str(s)).lower()
    s = re.sub(r"(โรงแรม|hotel)", " ", s, flags=re.IGNORECASE)
    s = re.sub(r"กรุงเทพ(?:มหานคร|ฯ)?", " ", s)
    s = re.sub(r"[,\|\(\)\[\]{}\"'’`•·/.:;#&\-–—_+*^@!$?~]", " ", s)
    s = re.sub(r"\s+", "", s)
    return s

def normalize_addr(s: str) -> str:
    if pd.isna(s): return ""
    s = unicodedata.normalize("NFKC", str(s)).lower().strip()
    rep = {"กทม.": "กรุงเทพมหานคร", "กรุงเทพฯ": "กรุงเทพมหานคร", "จ.": "", "เขต ": "", "แขวง ": "", "ถ.": "", "ถนน ": "", "บ้านเลขที่": ""}
    for k,v in rep.items(): s = s.replace(k, v)
    s = re.sub(r"[\"'’`•·\|\[\]{}#@!$^*?_+=:;,]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return "".join(s.split())

trip["Trip_Name_n"]    = trip["Trip_Name"].map(normalize_name)
trip["Trip_Address_n"] = trip["Trip_Address"].map(normalize_addr)
hotels["Hotel_Name_n"]    = hotels["Hotel_Name"].map(normalize_name)
hotels["Hotel_Address_n"] = hotels["Hotel_Address"].map(normalize_addr)

trip["Trip_Key"]    = (trip["Trip_Name_n"]    + " " + trip["Trip_Address_n"]).str.strip()
hotels["Hotel_Key"] = (hotels["Hotel_Name_n"] + " " + hotels["Hotel_Address_n"]).str.strip()

# Generate embeddings for keys and filter out texts that failed to embed
emb_trip_key, valid_trip_keys = ollama_embed(trip["Trip_Key"].tolist())
emb_hotel_key, valid_hotel_keys = ollama_embed(hotels["Hotel_Key"].tolist())

# Filter the original dataframes based on successfully embedded keys
trip_filtered = trip[trip["Trip_Key"].isin(valid_trip_keys)].reset_index(drop=True)
hotels_filtered = hotels[hotels["Hotel_Key"].isin(valid_hotel_keys)].reset_index(drop=True)

# Re-generate embeddings for name and address using the filtered data
emb_trip_name, _ = ollama_embed(trip_filtered["Trip_Name_n"].tolist())
emb_trip_addr, _ = ollama_embed(trip_filtered["Trip_Address_n"].tolist())
emb_hotel_name, _ = ollama_embed(hotels_filtered["Hotel_Name_n"].tolist())
emb_hotel_addr, _ = ollama_embed(hotels_filtered["Hotel_Address_n"].tolist())


# Perform Nearest Neighbors search only if embeddings have features
if emb_hotel_key.shape[1] > 0 and emb_trip_key.shape[1] > 0:
    nn = NearestNeighbors(n_neighbors=1, metric="cosine")
    nn.fit(emb_hotel_key)
    dist, idx = nn.kneighbors(emb_trip_key, return_distance=True)
    sim_key  = 1.0 - dist.ravel()
    best_idx = idx.ravel()

    # Ensure indices are within bounds of the filtered hotels DataFrame
    valid_indices = idx.ravel() < len(hotels_filtered)
    sim_key = sim_key[valid_indices]
    best_idx = idx.ravel()[valid_indices]
    trip_filtered = trip_filtered[valid_indices].reset_index(drop=True)
    emb_trip_name = emb_trip_name[valid_indices]
    emb_trip_addr = emb_trip_addr[valid_indices]


    # Ensure the dimensions match for einsum
    if emb_trip_name.shape[0] == emb_hotel_name[best_idx].shape[0] and \
       emb_trip_addr.shape[0] == emb_hotel_addr[best_idx].shape[0]:

        best_name = np.einsum("ij,ij->i", emb_trip_name, emb_hotel_name[best_idx])
        best_addr = np.einsum("ij,ij->i", emb_trip_addr, emb_hotel_addr[best_idx])

        KEY_THRESH = 0.95
        is_match = (sim_key >= KEY_THRESH)

        # Generate results
        matched_hotels = hotels_filtered.iloc[best_idx].reset_index(drop=True)

        trip_match = trip_filtered.copy()
        trip_match["Match_Hotel_Name"]    = np.where(is_match, matched_hotels["Hotel_Name"].values, np.nan)
        trip_match["Match_Hotel_Address"] = np.where(is_match, matched_hotels["Hotel_Address"].values, np.nan)
        trip_match["Match_Hotel_Ref"]     = np.where(is_match, matched_hotels.get("Hotel_Ref", pd.Series([np.nan]*len(trip_match))).values, np.nan)

        trip_match["sim_name"] = best_name
        trip_match["sim_addr"] = best_addr
        trip_match["sim_key"]  = sim_key
        trip_match["is_match"] = is_match

        trip_match["Best_Hotel_Name"]    = matched_hotels["Hotel_Name"].values
        trip_match["Best_Hotel_Address"] = matched_hotels["Hotel_Address"].values
        trip_match["Best_Hotel_Ref"]     = matched_hotels.get("Hotel_Ref", pd.Series([np.nan]*len(trip_match))).values

        # Display results for matched rows
        print(f"Matched {is_match.sum()}/{len(trip_match)} rows ({is_match.sum()/len(trip_match):.1%}) with KEY_THRESH={KEY_THRESH} (based on successfully embedded keys)")
        display(trip_match[is_match].head(20))

    else:
        print("Dimension mismatch after filtering. Cannot calculate similarities.")

else:
    print("Skipping Nearest Neighbors search due to empty embeddings after filtering.")

Ollama server is already running.
Pulling bge-m3...
bge-m3 is already pulled.


Embedding with bge-m3:   0%|          | 0/2665 [00:00<?, ?it/s]

Embedding with bge-m3:   0%|          | 0/1400 [00:00<?, ?it/s]

Embedding with bge-m3: 0it [00:00, ?it/s]

Embedding with bge-m3: 0it [00:00, ?it/s]

Embedding with bge-m3: 0it [00:00, ?it/s]

Embedding with bge-m3: 0it [00:00, ?it/s]

Skipping Nearest Neighbors search due to empty embeddings after filtering.


## Method 3: NER

In [8]:
# === Extract structured address for `hotel` table via Ollama ===
import re, json, pandas as pd, requests
from requests.adapters import HTTPAdapter, Retry
from tqdm.auto import tqdm # Import tqdm
import time
import subprocess

OLLAMA_URL = "http://127.0.0.1:11434"
ADDR_MODEL = "scb10x/typhoon2.1-gemma3-4b:latest"

# robust HTTP session (auto-retry)
_session = requests.Session()
_session.mount("http://", HTTPAdapter(max_retries=Retry(total=5, backoff_factor=0.5,
                                                        status_forcelist=[429,500,502,503,504])))

_PROMPT = """คุณเป็นตัวแยกโครงสร้างที่อยู่ภาษาไทย/อังกฤษ จงสกัดองค์ประกอบให้เป็น JSON เท่านั้น
อินพุต (อาจไม่มีเว้นวรรค/มีคำย่อ):
"{address}"

ให้ออกค่าเป็นออบเจ็กต์ JSON เดียวตามสChemต่อไปนี้:
{{
  "house_no": "",        // เลขที่บ้าน/อาคาร เช่น "178" หรือ "71/1" (เน้นดึงเลขที่อยู่ตัวแรก/ชุดแรก)
  "building": "",        // ชื่อตึก/โครงการ ถ้ามี
  "soi": "",             // ซอย/ตรอก
  "street": "",          // ถนน เช่น "สุรวงศ์"
  "subdistrict": "",     // แขวง/ตำบล เช่น "สี่พระยา"
  "district": "",        // เขต/อำเภอ เช่น "บางรัก"
  "province": "",        // จังหวัด เช่น "กรุงเทพมหานคร" (ถ้าอยู่ในกรุงเทพฯ ให้ใส่ "กรุงเทพมหานคร")
  "postcode": ""         // รหัสไปรษณีย์ 5 หลัก ถ้ามี
}}

กติกา:
- รวมคำย่อให้เป็นรูปมาตรฐาน เช่น "ถ./ถนน", "แขวง", "เขต", "จ./จังหวัด", "กทม./กรุงเทพฯ" → "กรุงเทพมหานคร"
- ถ้าไม่พบให้ใส่ค่าว่างให้เติมตามให้ถูกต้อง
- ตอบ JSON อย่างเดียว ไม่พูดคำนอก JSON
- ยืนยันว่าอยู่ในกรุงเทพมหานคร ประเทศไทย หากพบข้อมูลที่บ่งชี้
"""

def _json_block(text: str):
    s, e = text.find("{"), text.rfind("}")
    if s == -1 or e == -1 or e <= s: return None
    try:
        return json.loads(text[s:e+1])
    except json.JSONDecodeError:
        return None

def _extract_one(addr_text: str, timeout=45) -> dict:
    if not isinstance(addr_text, str) or not addr_text.strip():
        return {k:"" for k in ["house_no","building","soi","street","subdistrict","district","province","postcode"]}
    payload = {
        "model": ADDR_MODEL,
        "prompt": _PROMPT.format(address=addr_text.strip()),
        "stream": False,
        "options": {"temperature": 0.0}
        # "format": "json"   # ถ้า Ollama เวอร์ชันคุณรองรับ เปิดบรรทัดนี้ได้ จะช่วยให้ JSON เป๊ะขึ้น
    }
    try:
        r = _session.post(f"{OLLAMA_URL}/api/generate", json=payload, timeout=timeout)
        r.raise_for_status()
        resp = r.json().get("response","").strip()
        data = _json_block(resp)
        if not isinstance(data, dict):
             print(f"Warning: LLM did not return valid JSON for text: {addr_text[:50]}... Response: {resp[:100]}...")
             # fallback ว่าง ถ้า LLM ไม่คืน JSON ที่ถูกต้อง
             data = {"house_no":"","building":"","soi":"","street":"","subdistrict":"",
                     "district":"","province":"","postcode":""}
    except requests.exceptions.RequestException as e:
        print(f"Error calling Ollama API for text: {addr_text[:50]}... Error: {e}")
        # fallback ว่าง ถ้า LLM ล้มเหลว
        data = {"house_no":"","building":"","soi":"","street":"","subdistrict":"",
                "district":"","province":"","postcode":""}

    # ensure keys exist and are strings
    out = {k: (data.get(k) or "").strip() for k in
           ["house_no","building","soi","street","subdistrict","district","province","postcode"]}
    return out

def extract_addresses_hotel(hotel_df: pd.DataFrame, addr_col="Hotel_Address") -> pd.DataFrame:
    """Return a new DataFrame with structured address columns appended."""
    if addr_col not in hotel_df.columns:
        raise ValueError(f"Column `{addr_col}` not found in hotel_df")

    # Ensure Ollama server is running and model is pulled before processing
    try:
        subprocess.run("ollama list", shell=True, check=True, capture_output=True, text=True)
        print("Ollama server is running.")
    except subprocess.CalledProcessError:
        print("Starting Ollama server in background...")
        get_ipython().system_raw('nohup ollama serve > ollama_serve.log 2>&1 &')
        time.sleep(10) # Give server time to start
        try:
            subprocess.run("ollama list", shell=True, check=True, capture_output=True, text=True)
            print("Ollama server started successfully.")
        except subprocess.CalledProcessError:
            print("[FATAL ERROR] Ollama server failed to start. Address extraction will likely fail.")
            # Decide how to handle fatal error - maybe return df with empty address columns?
            df = hotel_df.copy()
            empty_parts = pd.DataFrame([{f"{addr_col}__{k}": "" for k in ["house_no","building","soi","street","subdistrict","district","province","postcode"]}] * len(df))
            return pd.concat([df.reset_index(drop=True), empty_parts], axis=1)

    print(f"Pulling {ADDR_MODEL}...")
    try:
        subprocess.run(f"ollama list | grep {ADDR_MODEL}", shell=True, check=True, capture_output=True, text=True)
        print(f"{ADDR_MODEL} is already pulled.")
    except subprocess.CalledProcessError:
        print(f"Pulling {ADDR_MODEL}...")
        pull_process = subprocess.run(f"ollama pull {ADDR_MODEL}", shell=True, capture_output=True, text=True)
        if pull_process.returncode != 0:
            print(f"[ERROR] Failed to pull {ADDR_MODEL}. Address extraction may not work.")
        else:
             print(f"Pulled {ADDR_MODEL}.")


    df = hotel_df.copy()
    texts = df[addr_col].fillna("").astype(str)

    # de-dup calls -> map unique address -> parsed JSON
    uniq = texts.drop_duplicates()
    parsed_map = {}
    # Add tqdm to the loop over unique addresses
    for t in tqdm(uniq, desc="LLM extract addresses"):
        parsed_map[t] = _extract_one(t)

    parts = texts.map(parsed_map).apply(pd.Series)
    parts.columns = [f"{addr_col}__{c}" for c in parts.columns]
    return pd.concat([df.reset_index(drop=True), parts.reset_index(drop=True)], axis=1)

# ---------- Example usage ----------
hotels = hotel.copy()  # สมมติ DataFrame ชื่อ `hotel` มีคอลัมน์ Hotel_Address
# Test with only the first 10 rows
hotels_ex = extract_addresses_hotel(hotels.head(10), addr_col="Hotel_Address")
display(hotels_ex.head(10)) # Display all 10 rows for verification

Ollama server is running.
Pulling scb10x/typhoon2.1-gemma3-4b:latest...
scb10x/typhoon2.1-gemma3-4b:latest is already pulled.


LLM extract addresses:   0%|          | 0/10 [00:00<?, ?it/s]

,Hotel_Ref,Hotel_Name,Hotel_Address,N,Hotel_Address__house_no,Hotel_Address__building,Hotel_Address__soi,Hotel_Address__street,Hotel_Address__subdistrict,Hotel_Address__district,Hotel_Address__province,Hotel_Address__postcode
0,101/2563,โรงแรม สุทธิสารพ้อยส์,45 ซ.ยิ้มอุปถัมภ์ แขวงรัชดาภิเษก เขตดินแดง จ.ก...,42,45,,ยิ้มอุปถัมภ์,,รัชดาภิเษก,ดินแดง,กรุงเทพมหานคร,10400
1,136/2563,โรงแรม คณาบูทีค 2,"3,5,7,9,11 ซ.จรัญสนิทวงศ์ 57/2 แขวงบางบำหรุ เข...",24,3,,57/2,จรัญสนิทวงศ์,บางบำหรุ,บางพลัด,กรุงเทพมหานคร,10700
2,152/2563,วัลย์ลดา เพลส,71/1 ซ.รามคำแหง 53 (จันทร์ศรีชวาลา) แขวงพลับพล...,50,71/1,จันทร์ศรีชวาลา,ซ.รามคำแหง 53,,พลับพลา,วังทองหลาง,กรุงเทพมหานคร,10310
3,214/2563,โรงแรม นิวสยาม พาเลซวิลล์ 2,56 ตรอกโรงไหม แขวงชนะสงคราม เขตพระนคร จ.กรุงเท...,48,56,,ตรอกโรงไหม,,ชนะสงคราม,พระนคร,กรุงเทพมหานคร,10200
4,215/2563,โรงแรม นิวสยาม พาเลซวิลล์ 3,58 ตรอกโรงไหม แขวงชนะสงคราม เขตพระนคร จ.กรุงเท...,48,58,,ตรอกโรงไหม,,ชนะสงคราม,พระนคร,กรุงเทพมหานคร,10200
5,6/2564,นิววังทอง,304 ถ.มหาชัย แขวงสำราญราษฎร์ เขตพระนคร จ.กรุงเ...,46,304,,,มหาชัย,สำราญราษฎร์,พระนคร,กรุงเทพมหานคร,
6,7/2564,39,6 ถ.บูรณศาสตร์ แขวงศาลเจ้าพ่อเสือ เขตพระนคร จ....,49,,,6,ถนนบูรณศาสตร์,ศาลเจ้าพ่อเสือ,พระนคร,กรุงเทพมหานคร,
7,14/2564,นครพิงค์,9/1 ซ.วรพงษ์ แขวงบ้านพานถม เขตพระนคร จ.กรุงเทพ...,118,9/1,,ซ.วรพงษ์,,บ้านพานถม,พระนคร,กรุงเทพมหานคร,10200
8,17/2564,พัฒนา(28),233 ถ.สามเสน แขวงวัดสามพระยา เขตพระนคร จ.กรุงเ...,79,233,,,สามเสน,วัดสามพระยา,พระนคร,กรุงเทพมหานคร,
9,32/2564,เค.ที,115/6 ถ.ประชาธิปไตย แขวงบางขุนพรหม เขตพระนคร จ...,105,115,,ถ.ประชาธิปไตย,ประชาธิปไตย,บางขุนพรหม,พระนคร,กรุงเทพมหานคร,10200


In [11]:

# Test with only the first 10 rows
OTA_df_ex = extract_addresses_hotel(OTA_df.head(10), addr_col="address")
display(OTA_df_ex.head(10)) # Display all 10 rows for verification

Ollama server is running.
Pulling scb10x/typhoon2.1-gemma3-4b:latest...
scb10x/typhoon2.1-gemma3-4b:latest is already pulled.


LLM extract addresses:   0%|          | 0/10 [00:00<?, ?it/s]

,name,name href,address,OTA,address__house_no,address__building,address__soi,address__street,address__subdistrict,address__district,address__province,address__postcode
0,นาซ่า กรุงเทพฯ,https://th.trip.com/hotels/detail/?cityId=359&...,"44 ถ. สุขุมวิท 71 แขวงสวนหลวง, เขตสวนหลวง, กรุ...",trip_com,44,,,สุขุมวิท,สวนหลวง,สวนหลวง,กรุงเทพมหานคร,10250
1,โรงแรมอีสติน แกรนด์ พญาไท,https://th.trip.com/hotels/detail/?cityId=359&...,"18 ถ.พญาไท ทุ่งพญาไท, เขตราชเทวี, กรุงเทพฯ, 10400",trip_com,18,,ถ.,พญาไท,ทุ่งพญาไท,ราชเทวี,กรุงเทพมหานคร,10400
2,โรงแรมเอเชียกรุงเทพ,https://th.trip.com/hotels/detail/?cityId=359&...,"269 ถ.พ, เขตราชเทวี, กรุงเทพฯ, 10400",trip_com,269,,ถ.,พ,ราชเทวี,กรุงเทพฯ,กรุงเทพมหานคร,10400
3,โรงแรมทวิน ทาวเวอร์,https://th.trip.com/hotels/detail/?cityId=359&...,"88 รองเมือง ปทุมวัน, ถ.พระราม 6, เขตปทุมวัน, ก...",trip_com,88,,ถ.พระราม 6,พระราม 6,ปทุมวัน,ปทุมวัน,กรุงเทพมหานคร,10330
4,โรงแรมเดอะ เบอร์เคลีย์ ประตูน้ำ,https://th.trip.com/hotels/detail/?cityId=359&...,"559 ราชปรารภ, มักกะสัน ราชเทวี, เขตราชเทวี, กร...",trip_com,559,,,ราชปรารภ,ราชเทวี,ราชเทวี,กรุงเทพมหานคร,10400
5,12 เดอะ เรสซิเดนซ์ โฮเทลแอนด์อพาร์ตเมนต์,https://th.trip.com/hotels/detail/?cityId=359&...,"22 ซ.สรณคมน์ 12 แขวงสีกัน, เขตดอนเมือง, กรุงเท...",trip_com,22,,ซ.สรณคมน์,,สีกัน,ดอนเมือง,กรุงเทพมหานคร,10210
6,ไอบิส สไตล์ กรุงเทพ สีลม,https://th.trip.com/hotels/detail/?cityId=359&...,"289 ถ.สีลม, เขตบางรัก, กรุงเทพฯ, 10500",trip_com,289,,ถ.,สีลม,บางรัก,บางรัก,กรุงเทพมหานคร,10500
7,AETAS ลุมพินี,https://th.trip.com/hotels/detail/?cityId=359&...,"1030 4 ถ. พระรามที่ ๔ แขวงท, เขตสาทร, กรุงเทพฯ...",trip_com,1030,,4,ถนนพระรามที่ ๔,ท,สาทร,กรุงเทพมหานคร,10120
8,เมอร์เคียว กรุงเทพ สุรวงศ์,https://th.trip.com/hotels/detail/?cityId=359&...,"222 ถนนสุรวงศ์ แขวงสี่พระยา, เขตบางรัก, กรุงเท...",trip_com,222,,,สุรวงศ์,สี่พระยา,บางรัก,กรุงเทพมหานคร,10500
9,ชาเทรียม แกรนด์ กรุงเทพฯ,https://th.trip.com/hotels/detail/?cityId=359&...,"728 ถ., เขตราชเทวี, กรุงเทพฯ, 10400",trip_com,728,,ถ.,,ราชเทวี,กรุงเทพฯ,กรุงเทพมหานคร,10400
